In [1]:
from sklearn import datasets
from sklearn.cross_validation import train_test_split
import numpy as np
import math
import operator
import tensorflow as tf
import pandas as pd
from collections import Counter

C:\Users\123\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\123\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#1、将鸢尾花数据8:2的比例划分成训练集和验证集
iris=datasets.load_iris()#加载鸢尾花数据集
data=iris.data
target=iris.target
x_tr, x_te, y_tr, y_te = train_test_split(data,target,test_size = 0.2,random_state=1)
x_train=tf.constant(x_tr,dtype=tf.float32)
x_test=tf.constant(x_te,dtype=tf.float32)
y_train=tf.constant(y_tr,dtype=tf.float32)
y_test=tf.constant(y_te,dtype=tf.float32)

In [3]:
#变量初始化
init=tf.global_variables_initializer()
#2.1 得到未知类别的样本的前K个近邻
def getNeighbors(trainingSet,testInstance,k):
    distances=[]
    neighbors=[]
    #计算欧式距离
    with tf.Session() as sess:
        sess.run(init)
        for x in range(trainingSet.shape[0]):
            dist=sess.run(tf.norm(trainingSet[x]-testInstance))
            distances.append((dist,x))
    distances.sort(key=operator.itemgetter(0))
    for x in range(k):
        neighbors.append(distances[x][1])
    return neighbors

In [4]:
#2、2采用少数服从多数的原则决出k个近邻中出现次数最多的
def getResponse(neighbors):
    classVotes =[]
    for x in range(len(neighbors)):
        response = y_train[neighbors[x]]
        classVotes.append(response)
        top=Counter(classVotes).most_common(1)
    return top[0][0]

In [5]:
#3训练模型
def main():
    predictions=[]
    k=3
    for x in range(x_test.shape[0]):
        neighbors=getNeighbors(x_train,x_test[x],k)
        res=getResponse(neighbors)
        with tf.Session() as sess:
            result=sess.run(res)
        predictions.append(result)
    return predictions

In [6]:
predictions=main()

In [7]:
#4、验证模型
#正确率函数
def getAccuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x] == predictions[x]:
            correct =correct+ 1
    return (correct/float(len(testSet)))*100.0

In [8]:
accuracy=getAccuracy(y_te,predictions)
print('Accuracy:'+repr(accuracy)+'%')

Accuracy:100.0%
